<a href="https://colab.research.google.com/github/soichi-fujiwara/jupyter-notebook/blob/master/SalesMail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install mecab-python3

     |████████████████████████████████| 15.9MB 4.6MB/s 


In [22]:
#======================================================
#単価スキップワード削除
#======================================================
str_wk = '【金　額】60万'

del_word = ["【", "】","(",")","（","）"," ","　"]

tanka_base = '単価'
tanka_yuragi = ["単金", "金額"]

#削除
for word in del_word:
  str_wk = str_wk.replace(word,"")

  #ゆらぎ統一
for tanka in tanka_yuragi:
  str_wk = str_wk.replace(tanka,tanka_base)

print(str_wk)


単価60万


In [0]:
import MeCab


#---------------------------------------------------------------------
#1.クリーニング処理
#(テキスト中のノイズを削除)
#---------------------------------------------------------------------


#---------------------------------------------------------------------
#2.単語の正規化
#(文字種の統一や大文字･小文字を揃える)
#---------------------------------------------------------------------


#---------------------------------------------------------------------
#3.文章の単語分割
#---------------------------------------------------------------------
tagger = MeCab.Tagger("-Owakati")
sample = "今日は天気が良いですね"

print(tagger.parse(sample))

#---------------------------------------------------------------------
#4.ストップワード除去
#---------------------------------------------------------------------


#---------------------------------------------------------------------
#5.単語のベクトル化
#---------------------------------------------------------------------



今日 は 天気 が 良い です ね 

